In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [ ]:
!ngrok config add-authtoken 2vXzaCl1lJSbbUgJcyvo59mKdfz_5KfjgXA4Svr2tcnes6XTT

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from transformers import AutoTokenizer
# from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import uvicorn
from pyngrok import ngrok
import threading
import time


In [ ]:
!pkill -f ngrok || true
time.sleep(2)

^C


In [ ]:
nest_asyncio.apply()

In [ ]:
# Create FastAPI app
app = FastAPI(
    title="CyberQwen2.5-7B API",
    description="API for querying the fine-tuned CyberQwen2.5-7B model",
    version="1.0.0"
)

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
# Global variables for model and tokenizer
model = None
tokenizer = None

print("Loading model and tokenizer...")
# Load using standard transformers library
tokenizer = AutoTokenizer.from_pretrained("Tiamz/CyberQwen2.5-7B")
model = AutoModelForCausalLM.from_pretrained(
    "Tiamz/CyberQwen2.5-7B",
    torch_dtype="auto",  # Use half precision for GPU memory efficiency
    device_map="auto",
    offload_folder="offload" # Let transformers handle device placement
)
print("Model loaded successfully")

Loading model and tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Model loaded successfully


In [ ]:
class QueryRequest(BaseModel):
    question: str
    max_tokens: int = 256

# Alpaca template for formatting inputs
alpaca_template = """Below is an instruction that describes a task.
Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

def format_response(generated_text):
    try:
        response = generated_text.split("### Response:")[1].strip()
        response = response.split("<|endoftext|>")[0].strip()
        return response
    except:
        return generated_text

In [ ]:
@app.post("/query")
async def query_model(request: QueryRequest):

    instruction = alpaca_template.format(request.question, "")


    inputs = tokenizer([instruction], return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=request.max_tokens,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id,
    )


    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    formatted_response = format_response(response_text)

    return {"response": formatted_response}


@app.get("/health")
async def health_check():
    return {"status": "healthy", "model_loaded": model is not None}

# Create and start ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)
print(f"Public URL: {ngrok_tunnel.public_url}")

# Start the FastAPI server in a separate thread
def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

server_thread = threading.Thread(target=start_server)
server_thread.daemon = True
server_thread.start()

print("Server is running. Use the public URL to access the API.")
print("API endpoints:")
print(f"- Health check: {ngrok_tunnel.public_url}/health")
print(f"- Query endpoint: {ngrok_tunnel.public_url}/query (POST)")
print(f"- API documentation: {ngrok_tunnel.public_url}/docs")
print("\nPress Ctrl+C to stop (or interrupt the cell execution)")

while True:
    time.sleep(1)

INFO:     Started server process [1359]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://3499-34-87-10-104.ngrok-free.app
Server is running. Use the public URL to access the API.
API endpoints:
- Health check: https://3499-34-87-10-104.ngrok-free.app/health
- Query endpoint: https://3499-34-87-10-104.ngrok-free.app/query (POST)
- API documentation: https://3499-34-87-10-104.ngrok-free.app/docs

Press Ctrl+C to stop (or interrupt the cell execution)


INFO:     2c0f:2a80:aa8:9910:c170:d3f:65d0:ac81:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2c0f:2a80:aa8:9910:c170:d3f:65d0:ac81:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2c0f:2a80:aa8:9910:c170:d3f:65d0:ac81:0 - "POST /query HTTP/1.1" 200 OK
INFO:     2c0f:2a80:aa8:9910:c170:d3f:65d0:ac81:0 - "GET /query HTTP/1.1" 405 Method Not Allowed
INFO:     2c0f:2a80:aa8:9910:c170:d3f:65d0:ac81:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2c0f:2a80:aa8:9910:c170:d3f:65d0:ac81:0 - "GET /query/what%20is%20cybersecurity HTTP/1.1" 404 Not Found
INFO:     2c0f:2a80:aa8:9910:c170:d3f:65d0:ac81:0 - "GET /health HTTP/1.1" 200 OK
INFO:     2c0f:2a80:aa8:9910:c170:d3f:65d0:ac81:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2c0f:2a80:aa8:9910:c170:d3f:65d0:ac81:0 - "GET /openapi.json HTTP/1.1" 200 OK


KeyboardInterrupt: 